Script to test the python mzml loader 
==

In [90]:
import sys
sys.path.append('/Users/simon/git/lda/code/')

In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load data using pymzml

In [92]:
data_dir = '/Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/'

In [121]:
from ms2lda_feature_extraction import LoadMZML
input_set = [data_dir + 'Beer_3_T10_POS.mzML']
peaklist = data_dir + 'Beer_3_full1.csv'
loader = LoadMZML(peaklist = peaklist,isolation_window = 0.5, mz_tol = 5, rt_tol = 10.0)
ms1,ms2,metadata = loader.load_spectra(input_set)

Loading spectra from /Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_T10_POS.mzML
Found 6686 ms2 spectra, and 191083 individual ms2 objects
Loaded 7863 ms1 peaks from /Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_full1.csv
0
96.0443760275 387.897003174
96.0438958056 96.0448562494 377.897003174 397.897003174
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
Peaklist filtering results in 1601 documents


No. of MS1 peaks that can be matched to the peaklist

In [94]:
len(ms1)

1582

No. of MS2 peaks

In [95]:
len(ms2)

55785

### Filtering

*SR: not sure this step is necesary?*

Try to filter the pymzml results using the same filter criteria as in method 3's

In [88]:
# keep MS1 peaks with RT >3 and <21 mins
ms1 = filter(lambda x: x.rt>(3*60) and x.rt<(21*60), ms1)
ms2 = filter(lambda x: x[3].rt>(3*60) and x[3].rt<(21*60), ms2)

# keep MS2 peaks with intensity >5000
ms2 = filter(lambda x: x[2]>5000, ms2)

In [89]:
len(ms1), len(ms2)

(1459, 22665)

Now load method 3's results

In [19]:
import pandas as pd
from IPython.display import display, HTML

In [20]:
ms1_df = pd.read_csv(data_dir + 'Beer3_POS_method3_csv_full1_ms1.csv', 
                      index_col=0)
ms2_df = pd.read_csv(data_dir + 'Beer3_POS_method3_csv_full1_ms2.csv', 
                      index_col=0)

In [21]:
ms1_df.shape[0], ms2_df.shape[0]

(1289, 24287)

### Try to match the MS1 peaks from both results

In [122]:
from matching import MatchFeature, SimpleMatching

In [123]:
origin = 'pymzml'
left = []
for peak in ms1:
    f = MatchFeature(peak.mz, peak.rt, peak.intensity, origin)
    left.append(f)

In [124]:
origin = 'method_3'
right = []
for idx, row in ms1_df.iterrows():
    mz = row[['mz']].values[0]
    rt = row[['rt']].values[0]
    intensity = row[['intensity']].values[0]    
    f = MatchFeature(mz, rt, intensity, origin)
    right.append(f)

This does a matching (based on the m/z and RT only) for the MS1 peaks produced by the left (pyzml) and right (method-3) sides.

In [125]:
m = SimpleMatching()
matches = m.run(left, right, 2, 5)

Computing scores
Matching
1059 matched feature pairs
542 unmatched features from the first
230 unmatched features from the second


** These are out of date **

Justin said: the following big MS1 peaks should appear on both sides but they dont.

    found by R but not by Python:
    mz=267.126477, rt=633.804016, intensity=32300000.000000 origin=method_3 (3E7!)
    mz=278.123377, rt=542.280029, intensity=178000000.000000 origin=method_3 (1.5E8!)

    for the latter, Python found a similar mz at a different RT that could interfer? --> mz=278.123131, rt=595.984985, intensity=1340108.250000 origin=pymzml

    and the other way around:
    mz=401.172490, rt=205.735992, intensity=13729020.000000 origin=pymzml (1.2E7)
    mz=258.108396, rt=493.036987, intensity=12743071.000000 origin=pymzml (1.2E7)

In [126]:
for match in matches:
    items = list(match)
    if len(items) > 1: # a pair
#         print items[0], '\n', items[1]
        pass
    else:
        print items[0] # single thing
    print

In [131]:
from ms2lda_feature_extraction import LoadMZML
input_set = [data_dir + 'Beer_3_T10_POS.mzML']
peaklist = data_dir + 'Beer_3_full1.csv'
loader = LoadMZML(peaklist = peaklist,isolation_window = 0.5, mz_tol = 5.0, rt_tol = 10.0)
npms1,npms2,npmetadata = loader.load_spectra(input_set)

Loading spectra from /Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_T10_POS.mzML
Found 6686 ms2 spectra, and 191083 individual ms2 objects
Loaded 7863 ms1 peaks from /Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_full1.csv
0
120.047788873 277.70199585
120.047188634 120.048389112 267.70199585 287.70199585
120.047791905 359.399993896
120.047191667 120.048392144 349.399993896 369.399993896
500


KeyboardInterrupt: 

In [102]:
npms1 = sorted(npms1,key = lambda x: x.mz)

In [103]:
for peak in npms1:
    print peak.mz,peak.rt

71.0727920532 606.606
72.0807952881 751.539
72.0807952881 835.548
72.0807952881 894.137
72.0807952881 1114.3
72.0808029175 1132.09
72.0808029175 1167.64
72.0808105469 800.203
72.0808105469 816.277
72.0808105469 910.937
72.0808105469 1068.29
72.0808105469 1151.97
72.0808105469 1198.44
72.0808105469 1213.8
72.0808105469 1244.08
72.0808105469 1259.83
72.0808105469 1275.07
72.0808105469 1305.39
72.0808105469 1327.49
72.0808181763 767.866
72.0808181763 783.91
72.0808181763 1183.27
72.0808181763 1228.95
72.0808181763 1290.21
72.0808258057 990.619
72.0808334351 735.418
72.080871582 562.506
73.064781189 462.438
74.0599822998 119.22
74.0599899292 134.796
74.0600204468 150.452
74.0600204468 181.692
74.0600280762 103.433
74.0600585938 166.126
74.0964126587 958.649
74.0964126587 1161.7
74.0964126587 1193.98
74.0964126587 1337.64
74.0964202881 24.4965
74.0964202881 41.1069
74.0964202881 87.7469
74.0964202881 197.534
74.0964202881 973.922
74.0964202881 1036.13
74.0964202881 1076.57
74.0964202881 109

In [111]:
1e6*abs(70.065071106 - 70.06516152)/70.065071106

1.29042900508996